## 1. Imports & Data Load
Assumes you already ran `preprocessing.ipynb` and saved `X.npy`, `Y.npy`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, utils, callbacks
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')
print('Loaded:', X.shape, Y.shape)
# Add channel axis
X = X[..., np.newaxis]
num_classes = len(np.unique(Y))
print('Num classes:', num_classes)

## 2. Train/Val/Test Split & One-Hot Labels

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)
print('Shapes -> Train:', train_X.shape, 'Val:', val_X.shape, 'Test:', test_X.shape)
train_y_cat = utils.to_categorical(train_y, num_classes)
val_y_cat = utils.to_categorical(val_y, num_classes)
test_y_cat = utils.to_categorical(test_y, num_classes)

## 3. Model Architecture
Time-Frequency convolutions followed by temporal GRUs.

In [ ]:
freq_bins, time_frames, _ = train_X.shape[1:]
input_shape = (freq_bins, time_frames, 1)
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, (5,5), activation='relu', padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2,2))(x)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2,2))(x)
x = layers.Permute((2,1,3))(x)
t = layers.TimeDistributed(layers.Flatten())(x)
t = layers.GRU(128, return_sequences=True)(t)
t = layers.GRU(64)(t)
out = layers.Dense(num_classes, activation='softmax')(t)
model = models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 4. Training

In [ ]:
es = callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
history = model.fit(train_X, train_y_cat, validation_data=(val_X, val_y_cat), epochs=50, batch_size=32, callbacks=[es, reduce_lr])

## 5. Evaluation (Quick Metrics)
Detailed reporting moved to `performance_report.ipynb`.

In [ ]:
pred_proba = model.predict(test_X)
pred = np.argmax(pred_proba, axis=1)
acc = accuracy_score(test_y, pred)
f1 = f1_score(test_y, pred, average='weighted')
print('Test accuracy:', acc)
print('Test weighted F1:', f1)

## 6. Save Model

In [ ]:
MODEL_SAVE = 'eeg_personid_model.h5'
model.save(MODEL_SAVE)
print('Saved model to', MODEL_SAVE)